# Threshold based mapping


In [ ]:
import ee
import geemap
import numpy as np

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-vikramscience85')

### Define Functions

In [ ]:
# Function to mask pixels with low CS+ QA scores.
def maskLowQA(image):
    qaBand = 'cs'
    clearThreshold = 0.5
    mask = image.select(qaBand).gte(clearThreshold)
    return image.updateMask(mask)

# Function to add NDVI, time, and constant variables
def addNDVI(image):
    #Return the image with the added bands.
    ndvi = image.normalizedDifference(['B8', 'B4']).rename(['ndvi'])
    return image.addBands([ndvi])

def permWaterMask():
    permWater1 = ee.ImageCollection("ESA/WorldCover/v200")\
            .first()\
            .select('Map')\
            .eq(80)
    permWater2 = ee.ImageCollection("ESA/WorldCover/v100")\
            .first()\
            .select('Map')\
            .eq(80)
    return permWater1.Or(permWater2)

def settleMask():
    settle1 = ee.ImageCollection("ESA/WorldCover/v200")\
            .first()\
            .select('Map')\
            .eq(50)
    settle2 = ee.ImageCollection("ESA/WorldCover/v100")\
            .first()\
            .select('Map')\
            .eq(50)
    return settle1.Or(settle2)

def Prethreshold(img1, img2, imag3):
    bl = img1.lte(0.4).And(img2.lte(0.4).And(img3.lte(0.4)))
    return bl

def Postthreshold(img1, img2, imag3):
    ll = img1.gte(0.4).And(img2.gte(0.4).And(img3.gte(0.4)))
    return ll

### Define the time filter

In [ ]:
## Previous Year
Pi_date = '2022-01-01' # Initial date of interest (inclusive).
Pf_date = '2025-12-30' # Final date of interest (exclusive).

## Current Year
Ci_date = '2023-01-01' # Initial date of interest (inclusive).
Cf_date = '2023-12-30' # Final date of interest (exclusive).

In [ ]:
# Define satellite Image collection.
s2P = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
filteredP = s2P\
           .filter(ee.Filter.date(Pi_date, Pf_date))
           .median()

s2A = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
filteredC = s2A\
           .filter(ee.Filter.date(Ci_date, Cf_date))
           .median()
# Load the Cloud Score+ collection
csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
csPlusBands = csPlus.first().bandNames()

# We need to add Cloud Score + bands to each Sentinel-2
# image in the collection
# This is done using the linkCollection() function
filteredS2WithCs_P = filteredP.linkCollection(csPlus, csPlusBands)
filteredS2WithCs_C = filteredC.linkCollection(csPlus, csPlusBands)

filteredMaskedP = filteredS2WithCs_P\
    .map(maskLowQA)\
    .select('B.*')

filteredMaskedC = filteredS2WithCs_C\
    .map(maskLowQA)\
    .select('B.*')

filteredWithVariablesP = filteredMaskedP\
    .map(addNDVI)

filteredWithVariablesC = filteredMaskedC\
    .map(addNDVI)


ndviP = filteredWithVariablesP.select('ndvi')
ndviC = filteredWithVariablesC.select('ndvi')